In [ ]:
import cv2
import numpy as np

def segment_floor_plan_traditional(image_path):
    # 读取图像
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 二值化
    _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    
    # 形态学操作
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # 查找轮廓
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 创建空白掩码
    mask = np.zeros_like(image)
    
    # 定义颜色
    colors = [
        (255, 0, 0),    # 红色 - 客厅
        (0, 255, 0),    # 绿色 - 卧室
        (0, 0, 255),    # 蓝色 - 厨房
        (255, 255, 0),  # 黄色 - 卫生间
        (255, 0, 255),  # 紫色 - 阳台
        (0, 255, 255),  # 青色 - 餐厅
        (128, 0, 0),    # 深红 - 书房
        (0, 128, 0)     # 深绿 - 走廊
    ]
    
    # 根据面积大小分配功能区（简单示例，实际需要更复杂的逻辑）
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    for i, contour in enumerate(contours[:len(colors)]):
        cv2.drawContours(mask, [contour], -1, colors[i], thickness=cv2.FILLED)
    
    # 叠加结果
    result = cv2.addWeighted(image, 0.7, mask, 0.3, 0)
    
    return image, mask, result

# 使用示例
image_path = "floor_plan.jpg"
original, mask, result = segment_floor_plan_traditional(image_path)

# 显示结果
cv2.imshow("Original", original)
cv2.imshow("Segmentation Mask", mask)
cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 保存结果
cv2.imwrite("traditional_mask.png", mask)
cv2.imwrite("traditional_result.png", result)